# Challenge Five

In [1]:
import os
import logging
import vertexai
from typing import List, Dict, Any, Optional
from pathlib import Path
from dotenv import load_dotenv

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

logger = logging.getLogger(__name__)
logging.basicConfig(filename="app.log", level=logging.INFO)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GCP_BUCKET = os.getenv("GCP_BUCKET")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION, staging_bucket=GCP_BUCKET)

### Agents

In [3]:
from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.tools import google_search
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

search_agent = LlmAgent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="Use the Google Search tool to find information about a topic.",
    tools=[google_search],
    output_key="search_results",
)

critique_agent = LlmAgent(
    name="critique_agent",
    model="gemini-2.0-flash",
    instruction="Critique the search results stored in the state key 'search_results'.",
    output_key="critique_results",
)

refine_agent = LlmAgent(
    name="refine_agent",
    model="gemini-2.5-flash",
    instruction="Refine the search results based on the critique stored in the state key 'critique_results' using the Google Search tool",
    tools=[google_search],
    output_key="refined_results",
)

presentation_agent = LlmAgent(
    name="presentation_agent",
    model="gemini-2.0-flash",
    instruction="Take the state key 'refined_results' and format it so that it can be used as the final output to an end user.",
)

greeter_agent = Agent(
    name="greeter_agent",
    model="gemini-2.0-flash",
    instruction="Greet the user and ask for a topic to search, then pass to 'search_agent'.",
)

search_sequence = SequentialAgent(
    name="search_sequence",
    sub_agents=[greeter_agent, search_agent, critique_agent, refine_agent, presentation_agent],
)


### Agent Testing

In [4]:
app = reasoning_engines.AdkApp(
    agent=search_sequence,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="Give me the news highlights in the world of sports.",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

Here are the latest sports news highlights:

**Football (Soccer)**

*   **FIFA Club World Cup 2025:** The expanded tournament is underway in the US. Paris Saint-Germain (PSG) will face Chelsea in the final after thrashing Real Madrid 4-0.
*   **UEFA Women's Euro 2025:** This major tournament is taking place in Switzerland from July 2 to July 25.

**Basketball**

*   **NBA Free Agency:** LeBron James opted into his $52.6 million player option with the Lakers. James Harden re-signed with the Clippers, and Julius Randle will stay with the Timberwolves. Deandre Ayton will join the Lakers after a buyout, and Myles Turner moved to the Milwaukee Bucks. The Phoenix Suns traded Kevin Durant to the Houston Rockets and Kristaps Porziņģis was traded to the Atlanta Hawks.

**Tennis**

*   **Wimbledon 2025:** Jannik Sinner and Novak Djokovic are set for a semi-final clash in the men's singles. Carlos Alcaraz also secured his place in the semi-finals. On the women's side, Emma Raducanu lost to world No. 1 Aryna Sabalenka in the third round. Aryna Sabalenka and Amanda Anisimova have also reached the semi-finals.

**Cricket**

*   **India vs. England Women's T20I Series:** India leads the series 2-1. India secured a historic T20I series victory against England by winning the fourth T20I by six wickets.
*   **World Championship of Legends Cricket 2025:** This tournament featuring former players will be hosted across four UK cities from July 18 to August 2.
*   **Men's Cricket:** Finn Allen (New Zealand) is out of the tri-series in Zimbabwe due to injury.

**Golf**

*   **PGA Tour:** Brian Campbell won the John Deere Classic.

**Other Sports**

*   **MLB:** The All-Star Game is scheduled for July 15 in Atlanta.
*   **Women's Rugby World Cup 2025:** England is set to host this tournament.


### Agent Deployment

In [5]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    app,
    requirements=str(Path.cwd() / 'requirements.txt'),
    display_name="Search Result Agent"
)

In [6]:
for event in remote_agent.stream_query(
    user_id="agent-engine-test-user",
    message="Give me the news highlights in the world of sports."
):
    display(Markdown(event["content"]["parts"][0]["text"]))

Hello! I'm here to help you find information on any topic you're interested in. You're looking for sports news highlights, correct? I will forward the request to the search agent.


Here are some of the recent sports news highlights:

*   **Basketball:** Kristaps Porzingis is reportedly going to the Hawks in a 3-team trade agreement. Cooper Flagg is expected to become Mavericks' new main player as the expected No. 1 pick in the NBA draft. NBA owners have unanimously approved the $1.5B sale of the Wolves and WNBA's Lynx from Taylor to Lore-Rodriguez group.
*   **Club World Cup:** Players and fans are coping with high temperatures as a heat wave hits the US. Benfica beat Bayern Munich in scorching heat to win Group C. Paris Saint-Germain (PSG) thrashed Real Madrid to set up a final against Chelsea.
*   **Gold Cup:** Alexi Lalas' Top 10 Players at the Gold Cup has been revealed.
*   **MLB:** Cal Raleigh hit his 32nd homer of the season as the Mariners extended their lead over the Twins. Aaron Judge hit a solo homer as the Yankees grabbed the lead over the Reds.
*   **Cricket:** India Women's team secured their first T20 series win in England.
*   **Tennis:** Iga Swiatek reached a new high to set up a Wimbledon semi-final with Belinda Bencic. Jannik Sinner has reached the Wimbledon semi-finals.
*   **Other Sports:** Ben Askren lost 50 pounds after spending 45 days in a coma.


Okay, here's a critique of the sports news highlights provided:

**Strengths:**

*   **Variety of Sports:** The summary covers a decent range of sports including basketball, soccer (Club World Cup and Gold Cup), baseball (MLB), cricket, tennis, and a miscellaneous "other sports" section. This caters to a broader audience.
*   **Specific Examples:** Instead of just saying "MLB news," it provides specific examples of player achievements (Raleigh's homer, Judge's homer). This makes the highlights more engaging and informative.
*   **Timeliness (Implied):** The language suggests recent events ("is reportedly going," "extended their lead," "reached a new high").

**Weaknesses:**

*   **Lack of Date/Timestamp:**  The biggest issue is the lack of any indication of *when* these events occurred. "Recent" is subjective. Are these headlines from today? This week? Last month? Without a date or timeframe, the information is significantly less useful.
*   **Inconsistent Detail:** The level of detail varies. For some sports (e.g., Club World Cup), there's mention of teams and scores, while others (e.g., Gold Cup) only mention a list by one person.
*   **Vague "Other Sports" Category:** The Ben Askren story is a bit random and doesn't really fit into a news *highlights* category. While interesting, it feels out of place. Also "other sports" needs specification (e.g., MMA/fighting).
*   **Missing Context:** Some items lack sufficient context. For example, the Kristaps Porzingis item assumes the user knows who he is and the significance of the trade. A brief explanation of *why* this is a highlight would be helpful.
*   **Possible Bias/Geographic Focus:** It *seems* to lean towards US-centric sports (MLB, NBA) and may need to be broadened, depending on the intended audience.

**Suggestions for Improvement:**

*   **Prioritize Timeliness:**  Include a date or timeframe (e.g., "Sports Highlights - July 17, 2024").
*   **Add Context:** Briefly explain the significance of each highlight.
*   **Standardize Detail:** Aim for a consistent level of detail across all sports.
*   **Refine "Other Sports" Category:** Either remove it or include more relevant/significant news. Specify the sport the news is about.
*   **Consider Audience:** Tailor the sports covered based on the user's location or stated preferences (if available).
*   **Source Attribution:** Ideally, provide links to the original news sources.

In summary, the highlights provide a decent overview of sports news but are significantly hampered by the lack of date information and varying levels of detail. Adding context and refining the "other sports" category would also improve the user experience.


Here's a digest of recent sports news highlights as of July 10, 2025:

**Basketball (NBA)**

The NBA free agency period is in full swing with several significant moves. Kevin Durant has been officially traded to the Houston Rockets as part of a seven-team deal, with the Suns receiving Jalen Green, Dillon Brooks, and multiple draft picks in return. Kristaps Porziņģis has been traded to the Atlanta Hawks, and Jrue Holiday moved to the Portland Trailblazers as part of a Celtics refresh. LeBron James opted into his $52.6 million player option with the Lakers for the 2025-26 season, while James Harden re-signed with the Clippers and Julius Randle will remain with the Timberwolves. Nikola Jokić reportedly declined a contract extension with the Nuggets, though this is not a cause for immediate concern in Denver. Other notable extensions include Jaren Jackson Jr. signing a five-year, $240 million max deal with the Grizzlies, and Paolo Banchero agreeing to a max contract with the Magic worth up to $287 million. Deandre Ayton was acquired by the Lakers after a buyout from the Trail Blazers.

**Tennis (Wimbledon)**

Wimbledon 2025 is heading into its semi-finals. In the men's singles, Carlos Alcaraz, Jannik Sinner, Novak Djokovic, and Taylor Fritz have secured their spots. Djokovic reached his record 14th Wimbledon semi-final after a challenging match against Flavio Cobolli. On the women's side, Aryna Sabalenka, Amanda Anisimova, and Iga Swiatek are among the semi-finalists. Belinda Bencic also advanced to her second Grand Slam semi-final by defeating Mirra Andreeva, and will face Iga Swiatek.

**Baseball (MLB)**

The field for the 2025 Home Run Derby is nearing completion, with Tampa Bay Rays' Junior Caminero, Nationals' James Wood, and Twins' Byron Buxton confirmed as participants, joining Atlanta Braves' Ronald Acuña Jr. The Derby is scheduled for July 14 in Atlanta. As the July 31st trade deadline approaches, the Twins' Joe Ryan is being monitored by multiple teams, including the Red Sox. The Yankees are also reportedly looking to shore up their third base.

**Cricket**

The third Test match between India and England in the Anderson-Tendulkar Trophy is set to begin on July 10, 2025, at Lord's, with the series tied 1-1. India's vice-captain Rishabh Pant hinted at the possibility of playing three pacers and one or two spinners for the match. Jofra Archer is expected to return for England in the third Test. In women's T20 cricket, the Indian team lost a recent last-over thriller to England by five runs, though they still held a 2-1 series lead at that point. Separately, Finn Allen has been ruled out of an upcoming T20I tri-series in Zimbabwe due to a foot injury. Tim David has signed a new two-year deal with the Hobart Hurricanes in the Big Bash League. The World Championship of Legends Cricket 2025 is scheduled from July 18 to August 2, featuring retired cricket superstars.

**Formula 1**

Christian Horner has exited his role as CEO and Team Principal of Red Bull Racing with immediate effect, with Laurent Mekies taking over. Horner, who had been with the team for 20 years, stated his departure came as a "shock." This managerial reshuffle comes amidst a season where Red Bull Racing is currently fourth in the constructors' standings.

**Golf**

The PGA Tour is set to host the 2025 Genesis Scottish Open at The Renaissance Club in North Berwick, Scotland, starting on July 10. World No. 1 Scottie Scheffler is the favorite, with Rory McIlroy also being a top contender. On the LPGA Tour, the Amundi Evian Championship, the fourth major of the 2025 season, is underway in France, featuring top players like Nelly Korda, Mao Saigo, Maja Stark, and Minjee Lee.

**Soccer (Football)**

The Football Governance Bill has cleared the Commons in the UK, paving the way for an independent regulator for English soccer, aimed at ensuring clubs' sustainability and accountability to fans. The summer 2025 Premier League transfer window is active, with updates on various club signings and releases. FC Barcelona has announced the arrival of new goalkeeper Joan Garcia.

**Other Sports**

Indian wrestler Reetika Hooda has been provisionally suspended after failing a doping test following the Asian Championships.

Here's the latest from the world of sports (as of July 10, 2025):

**Basketball (NBA):** The NBA free agency period is buzzing! Key moves include Kevin Durant to the Houston Rockets, Kristaps Porziņģis to the Atlanta Hawks, and Jrue Holiday to the Portland Trailblazers. LeBron James is staying with the Lakers, James Harden re-signed with the Clippers, and Julius Randle remains with the Timberwolves. Nikola Jokić declined a contract extension with the Nuggets. Jaren Jackson Jr. extended with Grizzlies with a max deal, and Paolo Banchero agreed to a max contract with the Magic. Deandre Ayton joined the Lakers after a buyout from the Trail Blazers.

**Tennis (Wimbledon):** Wimbledon semi-finalists are set! On the men's side, it's Carlos Alcaraz, Jannik Sinner, Novak Djokovic, and Taylor Fritz. Djokovic just reached a record 14th Wimbledon semi-final. The women's semi-finalists include Aryna Sabalenka, Amanda Anisimova, and Iga Swiatek. Belinda Bencic also advanced and will face Iga Swiatek.

**Baseball (MLB):** The Home Run Derby field is almost complete! Participants include Tampa Bay Rays' Junior Caminero, Nationals' James Wood, Twins' Byron Buxton, and Atlanta Braves' Ronald Acuña Jr. The Derby is on July 14 in Atlanta. Keep an eye on the Twins' Joe Ryan as a potential trade target. The Yankees are also reportedly looking to add at third base.

**Cricket:** The third Test match between India and England starts today at Lord's, with the series tied 1-1. Jofra Archer is expected to return for England. In women's T20 cricket, India lost a close match to England but still holds a series lead. Finn Allen is out of an upcoming T20I tri-series. Tim David signed a new deal with the Hobart Hurricanes. The World Championship of Legends Cricket 2025 is scheduled for July 18 to August 2.

**Formula 1:** Christian Horner is out as CEO and Team Principal of Red Bull Racing. Laurent Mekies is taking over.

**Golf:** The Genesis Scottish Open starts today in Scotland, with Scottie Scheffler and Rory McIlroy as top contenders. The Amundi Evian Championship, is underway in France, featuring Nelly Korda, Mao Saigo, Maja Stark, and Minjee Lee.

**Soccer (Football):** The Football Governance Bill has passed in the UK, paving the way for an independent regulator for English soccer. FC Barcelona has announced the arrival of new goalkeeper Joan Garcia.

**Other Sports:** Indian wrestler Reetika Hooda has been provisionally suspended for doping.
